In [1]:
import pandas as pd
import os

from vocab import Vocab
from constants import *

# <span style="text-decoration: underline">Statistics</span>
This is the file where statistics are calculated

#### <u>Stat-1: Method to calculate % of visits that have higher than certain threshold % of "[unk]" tokens </u>

In [ ]:
def percentage_visits_with_high_unk(vocab: Vocab, threshold: float = 0.3, for_single_vsit: bool = False):
    visit_record_key = ["SUBJECT_ID", "HADM_ID", "ICD9_CODE", "ATC4"]
    visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, MULTI_VISIT_PKL))[visit_record_key[1:]]
    if for_single_vsit is True:
        visit = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, SINGLE_VISIT_PKL))[visit_record_key[1:]]
    total = visit.shape[0]
    def process(row):
        icd: list[str] = row["ICD9_CODE"]
        unk = 0
        for code in icd:
            if vocab.word2idx.get(code) == None:
                unk += 1
        if unk/len(icd) > threshold:
            print(row["HADM_ID"])
            return 1
    temp = visit.apply(process, axis=1)
    count: int = temp.loc[temp == 1].shape[0]
    return count/total